In [ ]:
from tensorflow.keras.applications import InceptionV3, VGG16, ResNet50, MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
import keras
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
import os
import random
import datetime
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
root_path="data/"
ngantuk_path=root_path+"/ngantuk/wajah"
sadar_path=root_path+"/sadar/wajah"
chekpoints_path = "Model CNN/no patience/Checkpoint"
model_path= "Model CNN/no patience"

In [ ]:
# insert all image in dataset folder to list
ngantuks = []
sadars = []
for filename in os.listdir(ngantuk_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
      image = cv2.imread(os.path.join(ngantuk_path, filename))
      ngantuks.append(image)
      print(filename)

for filename in os.listdir(sadar_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
      image = cv2.imread(os.path.join(sadar_path, filename))
      sadars.append(image)
      print(filename)

# preparing image

In [ ]:
rotates=[cv2.ROTATE_90_CLOCKWISE,cv2.ROTATE_90_COUNTERCLOCKWISE]
# resize and input data, and target
img_size=200
data=[]
hitung=0
target=[]
for img1 in ngantuks:
  img=img1.copy()
  try:
    resized=cv2.resize(img,(img_size,img_size))
    rotated=cv2.rotate(resized,rotates[random.randint(0,1)])
    fliped=cv2.flip(resized,random.randint(0,1))
    data.append(resized)
    target.append("ngantuk")
    hitung+=1
    data.append(rotated)
    target.append("ngantuk")
    hitung+=1
    data.append(fliped)
    target.append("ngantuk")
    hitung+=1
  except Exception as e:
    print('Exception:',e)

for img1 in sadars:
  try:
    img=img1.copy()
    resized=cv2.resize(img,(img_size,img_size))
    rotated=cv2.rotate(resized,rotates[random.randint(0,1)])
    fliped=cv2.flip(resized,random.randint(0,1))
    data.append(resized)
    target.append("sadar")
    hitung+=1
    data.append(rotated)
    target.append("sadar")
    hitung+=1
    data.append(fliped)
    target.append("sadar")
    hitung+=1
  except Exception as e:
    print('Exception:',e)

print(str(len(ngantuks)+len(sadars)))
print(str(hitung))

In [ ]:
# encode kategori (target) jadi angka
encoder = LabelEncoder()
target=encoder.fit_transform(target)
target = keras.utils.to_categorical(target, 2)
# split data
xtrain,xtest,ytrain,ytest=train_test_split(data,target,test_size=0.2,random_state=1)

# initiate

In [ ]:
img_size=200

In [ ]:
# MobileNetV2
base_model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
# InceptionV3
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
# VGG16
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
# ResNet50
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# mobilenet

In [ ]:
def combined_metric(y_true, y_pred):
  val_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
  val_acc = tf.keras.metrics.categorical_accuracy(y_true, y_pred)
  return val_loss * (1 - val_acc)  # Lower values indicate better performance

In [ ]:
print("start : "+str(datetime.datetime.now().time()))

In [ ]:
# code 1
# Freeze the base model agar bisa di compile
base_model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model_mobilenet.trainable = False
print("1")
model_mobilenet = base_model_mobilenet.output
model_mobilenet = GlobalAveragePooling2D()(model_mobilenet)
model_mobilenet = Dense(128, activation='relu')(model_mobilenet)
model_mobilenet = Dense(2, activation='softmax')(model_mobilenet)
model = tf.keras.models.Model(inputs=base_model_mobilenet.input, outputs=model_mobilenet)

print("2")
xtrain2 = np.array(xtrain.copy())
xtest2 = np.array(xtest.copy())
ytrain2 = np.array(ytrain.copy())
ytest2 = np.array(ytest.copy())
print("3")
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', combined_metric])

checkPoint = tf.keras.callbacks.ModelCheckpoint(filepath=chekpoints_path+"/cp_mobilenetv2.keras",verbose=1)

early_stopping = EarlyStopping(monitor='val_combined_metric', min_delta=0.00001, mode='min',restore_best_weights=True)

print("4")
model.fit(xtrain2, ytrain2, epochs=32, validation_data=(xtest2, ytest2),callbacks=checkPoint)

evaluation_results = model.evaluate(xtest2, ytest2)

# Access the accuracy from the results (index 1 corresponds to accuracy)
accuracy = evaluation_results[1]
print('Accuracy: {}'.format(accuracy))
print('loss: {}'.format(evaluation_results[0]))
print('metriks: {}'.format(evaluation_results[2]))

model.save(model_path+"/model_mobilenetv2.keras")

In [ ]:
print("mobileNet done : "+str(datetime.datetime.now().time()))


# inception

## tanpa transer learning

## dengan transfer learning

In [ ]:
model_mobilenet=keras.models.load_model(model_path+"model_mobilenetv2.keras")

In [ ]:
# code test
# Freeze the base model
print("1")
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model_inception.trainable = False
print("2")
xtrain2 = np.array(xtrain.copy())
xtest2 = np.array(xtest.copy())
ytrain2 = np.array(ytrain.copy())
ytest2 = np.array(ytest.copy())
print("a")

# Extract features using the pre-trained MobileNetV2 model
# Remove the last layer from model_mobilenet to get 4D output
model_mobilenet_notop = tf.keras.models.Model(inputs=model_mobilenet.input, outputs=model_mobilenet.layers[-2].output)
features_train = model_mobilenet_notop.predict(xtrain2)
features_test = model_mobilenet_notop.predict(xtest2)
print("b")

# Build the InceptionV3 model on top of extracted features
model_inception = Sequential()

# Reshape features to 4D if they are flattened
if features_train.ndim == 2:
    features_train = features_train.reshape(features_train.shape[0], 1, 1, features_train.shape[1])
    features_test = features_test.reshape(features_test.shape[0], 1, 1, features_test.shape[1])

print("c")

model_inception.add(GlobalAveragePooling2D(input_shape=features_train.shape[1:]))  # Now features_train.shape[1:] should be 4D
model_inception.add(Dense(128, activation='relu'))
model_inception.add(Dense(2, activation='softmax'))

# Compile the model
model_inception.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', combined_metric])

checkPoint = tf.keras.callbacks.ModelCheckpoint(filepath=chekpoints_path+"/cp_inception_tf.keras",verbose=1)

early_stopping = EarlyStopping(monitor='val_combined_metric', min_delta=0.00001, mode='min',restore_best_weights=True)
print("d")

# Train the model
model_inception.fit(features_train, ytrain2, epochs=32, validation_data=(features_test, ytest2), callbacks=checkPoint)

# Evaluate the model
evaluation_results = model_inception.evaluate(features_test, ytest2)

# Access the accuracy from the results (index 1 corresponds to accuracy)
accuracy = evaluation_results[1]
print('Accuracy: {}'.format(accuracy))
print('loss: {}'.format(evaluation_results[0]))
print('metriks: {}'.format(evaluation_results[2]))

model_inception.save(model_path+"/model_inception_tf.keras")

In [ ]:
print("inception 2 : "+str(datetime.datetime.now().time()))


# VGG

## transfer

In [ ]:
# Freeze the base model
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model_vgg16.trainable = False

xtrain2 = np.array(xtrain.copy())
xtest2 = np.array(xtest.copy())
ytrain2 = np.array(ytrain.copy())
ytest2 = np.array(ytest.copy())

# Extract features using the pre-trained MobileNetV2 model
# Remove the last layer from model_mobilenet to get 4D output
model_mobilenet_notop = tf.keras.models.Model(inputs=model_mobilenet.input, outputs=model_mobilenet.layers[-2].output)
features_train = model_mobilenet_notop.predict(xtrain2)
features_test = model_mobilenet_notop.predict(xtest2)

# Build the VGG16 model on top of extracted features
model_vgg16 = Sequential()

# Reshape features to 4D if they are flattened
if features_train.ndim == 2:
    features_train = features_train.reshape(features_train.shape[0], 1, 1, features_train.shape[1])
    features_test = features_test.reshape(features_test.shape[0], 1, 1, features_test.shape[1])

model_vgg16.add(GlobalAveragePooling2D(input_shape=features_train.shape[1:]))  # Now features_train.shape[1:] should be 4D
model_vgg16.add(Dense(128, activation='relu'))
model_vgg16.add(Dense(2, activation='softmax'))

# Compile the model
model_vgg16.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', combined_metric])

checkPoint = tf.keras.callbacks.ModelCheckpoint(filepath=chekpoints_path+"/cp_vgg16_tf.keras",verbose=1)

early_stopping = EarlyStopping(monitor='val_combined_metric', min_delta=0.00001, mode='min',restore_best_weights=True)

# Train the model
model_vgg16.fit(features_train, ytrain2, epochs=32, validation_data=(features_test, ytest2), callbacks=checkPoint)

# Evaluate the model
evaluation_results = model_vgg16.evaluate(features_test, ytest2)

# Access the accuracy from the results (index 1 corresponds to accuracy)
accuracy = evaluation_results[1]
print('Accuracy: {}'.format(accuracy))
print('loss: {}'.format(evaluation_results[0]))
print('metriks: {}'.format(evaluation_results[2]))

model_vgg16.save(model_path+"/model_vgg16_tf.keras")

# ResNet

## transfer


In [ ]:
# prompt: transfer learning resnet 50 from mobile net

# Freeze the base model
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model_resnet50.trainable = False

xtrain2 = np.array(xtrain.copy())
xtest2 = np.array(xtest.copy())
ytrain2 = np.array(ytrain.copy())
ytest2 = np.array(ytest.copy())

# Extract features using the pre-trained MobileNetV2 model
# Remove the last layer from model_mobilenet to get 4D output
model_mobilenet_notop = tf.keras.models.Model(inputs=model_mobilenet.input, outputs=model_mobilenet.layers[-2].output)
features_train = model_mobilenet_notop.predict(xtrain2)
features_test = model_mobilenet_notop.predict(xtest2)

# Build the ResNet50 model on top of extracted features
model_resnet50 = Sequential()

# Reshape features to 4D if they are flattened
if features_train.ndim == 2:
    features_train = features_train.reshape(features_train.shape[0], 1, 1, features_train.shape[1])
    features_test = features_test.reshape(features_test.shape[0], 1, 1, features_test.shape[1])

model_resnet50.add(GlobalAveragePooling2D(input_shape=features_train.shape[1:]))  # Now features_train.shape[1:] should be 4D
model_resnet50.add(Dense(128, activation='relu'))
model_resnet50.add(Dense(2, activation='softmax'))

# Compile the model
model_resnet50.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', combined_metric])

checkPoint = tf.keras.callbacks.ModelCheckpoint(filepath=chekpoints_path+"/cp_resnet50_tf.keras",verbose=1)

early_stopping = EarlyStopping(monitor='val_combined_metric', min_delta=0.00001, mode='min',restore_best_weights=True)

# Train the model
model_resnet50.fit(features_train, ytrain2, epochs=32, validation_data=(features_test, ytest2), callbacks=checkPoint)

# Evaluate the model
evaluation_results = model_resnet50.evaluate(features_test, ytest2)

# Access the accuracy from the results (index 1 corresponds to accuracy)
accuracy = evaluation_results[1]
print('Accuracy: {}'.format(accuracy))
print('loss: {}'.format(evaluation_results[0]))
print('metriks: {}'.format(evaluation_results[2]))

model_resnet50.save(model_path+"/model_resnet50_tf.keras")

In [ ]:
print("end : "+str(datetime.datetime.now().time()))

In [ ]:
model_resnet50 = tf.keras.models.load_model("Model CNN/no patience/model_resnet50_tf.keras")
model_mobilenet=tf.keras.models.load_model("Model CNN/no patience/model_mobilenetv2.keras")
model_inception=tf.keras.models.load_model("Model CNN/no patience/model_inception_tf.keras")
model_vgg=tf.keras.models.load_model("Model CNN/no patience/model_vgg16_tf.keras")

In [8]:
xtrain2 = np.array(xtrain.copy())
xtest2 = np.array(xtest.copy())
ytrain2 = np.array(ytrain.copy())
ytest2 = np.array(ytest.copy())
model_mobilenet_notop = tf.keras.models.Model(inputs=model_mobilenet.input, outputs=model_mobilenet.layers[-2].output)
features_train = model_mobilenet_notop.predict(xtrain2)
features_test = model_mobilenet_notop.predict(xtest2)
if features_train.ndim == 2:
    features_train = features_train.reshape(features_train.shape[0], 1, 1, features_train.shape[1])
    features_test = features_test.reshape(features_test.shape[0], 1, 1, features_test.shape[1])

1066/1066 ━━━━━━━━━━━━━━━━━━━━ 1527s 1s/step
267/267 ━━━━━━━━━━━━━━━━━━━━ 383s 1s/step


In [10]:
loss,acc=model_mobilenet.evaluate(xtest2, ytest2)
print("mobilenet")
print("loss: ",loss)
print("acc: ",acc)
print("")

print("vgg16")
loss,acc=model_vgg.evaluate(features_test, ytest2)
print("loss: ",loss)
print("acc: ",acc)
print("")

print("inception")
loss,acc=model_inception.evaluate(features_test, ytest2)
print("loss: ",loss)
print("acc: ",acc)
print("")

print("resnet")
loss,acc=model_resnet50.evaluate(features_test, ytest2)
print("loss: ",loss)
print("acc: ",acc)

267/267 ━━━━━━━━━━━━━━━━━━━━ 395s 1s/step - accuracy: 0.9947 - loss: 0.0326
mobilenet
loss:  0.03174535557627678
acc:  0.9942501783370972

vgg16
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.9948 - loss: 0.1196
loss:  0.12244947254657745
acc:  0.9941328167915344

inception
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 832us/step - accuracy: 0.9950 - loss: 0.1094
loss:  0.1076526790857315
acc:  0.9943675398826599

resnet
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 821us/step - accuracy: 0.9952 - loss: 0.0992
loss:  0.10059183090925217
acc:  0.9946022033691406


In [12]:
model_resnet50 = tf.keras.models.load_model("Model CNN/epoch 10/model_resnet50_tf.keras")
model_mobilenet=tf.keras.models.load_model("Model CNN/epoch 10/model_mobilenetv2.keras")
model_inception=tf.keras.models.load_model("Model CNN/epoch 10/model_inception_transfer.keras")
model_vgg=tf.keras.models.load_model("Model CNN/epoch 10/model_vgg16_tf.keras")

In [14]:
xtrain2 = np.array(xtrain.copy())
xtest2 = np.array(xtest.copy())
ytrain2 = np.array(ytrain.copy())
ytest2 = np.array(ytest.copy())
model_mobilenet_notop = tf.keras.models.Model(inputs=model_mobilenet.input, outputs=model_mobilenet.layers[-2].output)
features_train = model_mobilenet_notop.predict(xtrain2)
features_test = model_mobilenet_notop.predict(xtest2)
if features_train.ndim == 2:
    features_train = features_train.reshape(features_train.shape[0], 1, 1, features_train.shape[1])
    features_test = features_test.reshape(features_test.shape[0], 1, 1, features_test.shape[1])

1066/1066 ━━━━━━━━━━━━━━━━━━━━ 1415s 1s/step
267/267 ━━━━━━━━━━━━━━━━━━━━ 369s 1s/step


In [15]:
# epoch 10
loss,acc=model_mobilenet.evaluate(xtest2, ytest2)
print("mobilenet")
print("loss: ",loss)
print("acc: ",acc)
print("")

print("vgg16")
loss,acc=model_vgg.evaluate(features_test, ytest2)
print("loss: ",loss)
print("acc: ",acc)
print("")

print("inception")
loss,acc=model_inception.evaluate(features_test, ytest2)
print("loss: ",loss)
print("acc: ",acc)
print("")

print("resnet")
loss,acc=model_resnet50.evaluate(features_test, ytest2)
print("loss: ",loss)
print("acc: ",acc)

267/267 ━━━━━━━━━━━━━━━━━━━━ 374s 1s/step - accuracy: 0.9872 - loss: 0.0433
mobilenet
loss:  0.040294528007507324
acc:  0.9872095584869385

vgg16
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 742us/step - accuracy: 0.9944 - loss: 0.0217
loss:  0.022837607190012932
acc:  0.9936634302139282

inception
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 724us/step - accuracy: 0.9936 - loss: 0.0433
loss:  0.03844885155558586
acc:  0.9926073551177979

resnet
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 735us/step - accuracy: 0.9941 - loss: 0.0206
loss:  0.022178806364536285
acc:  0.9934287667274475
